<a href="https://colab.research.google.com/github/Dianxia-starco/fun-arduino-codes/blob/main/Chat_Bot_Cheet_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all required libraries for the chatbot
# LangChain: Framework for building LLM applications
# Google Generative AI: Python SDK for Gemini API
# Gradio: Creates web interface for the chatbot
# HuggingFace Hub: Optional for model deployment
!pip install -U langchain langchain-community langchain-google-genai
!pip install -U google-generativeai
!pip install -U gradio
!pip install -U huggingface_hub

In [ ]:
import os
import gradio as gr
import google.generativeai as genai

**How to get Google Gemini API Key?**

- Go to https://aistudio.google.com/app/api-keys
- Click "Create API Key"
- Copy the API Key for your use

In [ ]:
GEMINI_API_KEY="..."
genai.configure(api_key=GEMINI_API_KEY)


- Similar to Gemini Model we can also use HuggingFace Transformer Models.
- Reference links: https://python.langchain.com/docs/integrations/providers/huggingface , https://python.langchain.com/docs/integrations/llms/huggingface_hub.html



In [ ]:
# from langchain.llms import HuggingFacePipeline
# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",)

In [ ]:
# Initialize Gemini model
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Custom LLM wrapper for Gemini
class GeminiLLM:
    def __init__(self, model):
        self.model = model
        self.memory_history = []

    def predict(self, user_message):
        # Build conversation context
        full_prompt = "You are a helpful assistant to answer user queries.\n"
        for msg in self.memory_history:
            full_prompt += f"{msg}\n"
        full_prompt += f"User: {user_message}\nChatbot:"

        # Generate response
        response = self.model.generate_content(full_prompt)
        answer = response.text

        # Update memory
        self.memory_history.append(f"User: {user_message}")
        self.memory_history.append(f"Chatbot: {answer}")

        # Keep only last 10 exchanges
        if len(self.memory_history) > 20:
            self.memory_history = self.memory_history[-20:]

        return answer

llm_chain = GeminiLLM(gemini_model)

In [ ]:
def get_text_response(user_message,history):
    response = llm_chain.predict(user_message = user_message)
    return response

In [ ]:
demo = gr.ChatInterface(get_text_response, examples=["How are you doing?","What are your interests?","Which places do you like to visit?"])

In [ ]:
if __name__ == "__main__":
    demo.launch(debug=True) #To create a public link, set `share=True` in `launch()`. To enable errors and logs, set `debug=True` in `launch()`.

##**Publishing your code to Hugging Face**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
HUGGING_FACE_REPO_ID = "<Hugging Face User Name/Repo Name>"

**Adding Secret Variables in Hugging Face Account:**

- Open your Space
- Click on Settings Button
- Checkout to **Variables and secrets** section
- Create New Secrets

*Note*: Make sure to add your **google-generativeai_API_KEY** in Secret key

In [ ]:
%mkdir /content/ChatBotWithGemini
!wget -P  /content/ChatBotWithGemini/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWIthGeminiLangchain/app.py
!wget -P /content/ChatBotWithGemini/ https://s3.ap-south-1.amazonaws.com/cdn1.ccbp.in/GenAI-Workshop/ChatBotWIthGeminiLangchain/requirements.txt

In [ ]:
%cd /content/ChatBotWithGemini

api.upload_file(
    path_or_fileobj="./requirements.txt",
    path_in_repo="requirements.txt",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")

api.upload_file(
    path_or_fileobj="./app.py",
    path_in_repo="app.py",
    repo_id=HUGGING_FACE_REPO_ID,
    repo_type="space")